# Data Preparation Forecast Stores Revenue

Esta preparação de dados tem como objetivo, ter os dados em granularidade semanal e adicionar váriáveis exógenas aos nossos dados.

In [1]:
# Importar biblioteca
import pandas as pd
# Carregar a flatTable
flatTable = pd.read_csv("https://media.githubusercontent.com/media/E-man85/projectII/main/dataStaging/flatTable.csv")  
flatTable.shape

(8583762, 30)

In [2]:
# Selecionar dados da flatTable a serem usados
flatTable1 = flatTable.copy()
# Selecionar colunas
columns = ['store_id', 'city_code','date', 'revenue','latitude', 'longitude']
flatTable1= flatTable1[columns]
# Converter a coluna 'date' em objeto de data
flatTable1['date'] = pd.to_datetime(flatTable1['date'])
# Agrupar revenue por dia
flatTable1 = flatTable1.groupby(['store_id', 'city_code', 'date', 'latitude', 'longitude']).sum('revenue').reset_index()
flatTable1.head()

,store_id,city_code,date,latitude,longitude,revenue
0,S0002,Adana,2017-01-02,36.98636,35.325286,171.59
1,S0002,Adana,2017-01-03,36.98636,35.325286,178.10
2,S0002,Adana,2017-01-04,36.98636,35.325286,303.89
3,S0002,Adana,2017-01-05,36.98636,35.325286,274.65
4,S0002,Adana,2017-01-06,36.98636,35.325286,193.81


In [3]:
# Criar dicionario de dataframes por store_id
flatTable2 = flatTable1.copy()
# Criar uma lista das lojas únicas
stores = flatTable2['store_id'].unique().tolist()
# Criar um dicionário para armazenar os dataframes separados
df_dict = {}
# Loop para criar os dataframes separados
for store in stores:
    # Filtrar o dataframe com base no valor da loja
    df_store = flatTable2[flatTable2['store_id'] == store].copy()
    # Redefinir o índice do dataframe
    df_store = df_store.reset_index(drop=True)
    # Adicionar o dataframe ao dicionário usando o nome da store_id como chave
    df_dict[store] = df_store
# Ver elementos do dicionario
len(df_dict)

63

# Datas ausentes 
Verificar se em cada loja os registos têm date com frequência diária, e estando em falta, criar registo.

In [4]:
# Verificar datas ausentes tendo em conta a data min e max de date de cada store
store_missing_dates = []
# Loop para percorrer dataframes do dicionario
for df in df_dict.values():
# Verificar se a coluna 'date' tem alguma data em falta
    date_column = df['date']
    if date_column.diff().gt(pd.Timedelta(days=1)).any():
        store = df['store_id'].unique()[0]
        store_missing_dates.append(store)
        print(f"A coluna 'date' da loja {df['store_id'].unique()} tem datas em falta.")

A coluna 'date' da loja ['S0072'] tem datas em falta.
A coluna 'date' da loja ['S0136'] tem datas em falta.


In [5]:
# Inserir datas em falta
df_dict1 = df_dict.copy()
# Loop
for store in store_missing_dates:
    df = df_dict1[store] 
    latitude = df['latitude'].unique()
    longitude = df['longitude'].unique()
    city_code = df['city_code'].unique()
# Definir a coluna 'date' como o índice do dataframe
    df = df.set_index('date')
# Obter a data mínima e máxima do dataframe
    data_min = df.index.min()
    data_max = pd.to_datetime('2019-09-30')  # Definir data_max como '2019-09-30'
# Gerar o intervalo completo de datas
    intervalo_datas = pd.date_range(start=data_min, end=data_max).date
# Reindexar o dataframe com o intervalo completo de datas
    df = df.reindex(intervalo_datas)
# Verificar e preencher os valores ausentes
    df['store_id'].fillna(store, inplace=True)
    df['revenue'].fillna(0, inplace=True)
    df['latitude'].fillna(latitude[0], inplace = True)
    df['longitude'].fillna(longitude[0], inplace = True)
    df['city_code'].fillna(city_code[0],inplace = True)
# Reset do índice do dataframe
    df = df.reset_index()
    # Reordenar as colunas
    df = df[['store_id', 'city_code', 'date', 'latitude', 'longitude' , 'revenue']]
    df_dict1[store] = df
# Ver elementos do dicionario
len(df_dict1)

63

## Criação de variável temperature

A variável temperature é a temperatura média registada em cada dia, usando a localização da cidade onde está localizada

In [6]:
# Dados meteorológicos
# Juntar todos os DataFrames em um único DataFrame
df_dict2 = df_dict1.copy()
df_concatenado = pd.concat(df_dict2.values())
# Criar dimensão do tempo com city_code, latitude, longitude , min_date, max_date
dim_temp = df_concatenado.copy()
dim_temp = dim_temp[['date', 'city_code', 'latitude', 'longitude']]
dim_temp['date'] = pd.to_datetime(dim_temp['date'])
dim_temp = dim_temp.drop_duplicates()
# Obter as datas mínima e máxima
min_date = dim_temp['date'].min().to_pydatetime().date()
max_date = dim_temp['date'].max().to_pydatetime().date()
# Converter as datas para strings
min_date_str = min_date.strftime('%Y-%m-%d')
max_date_str = max_date.strftime('%Y-%m-%d')
# Remover a coluna 'date'
dim_temp = dim_temp.drop('date', axis=1)
# Adicionar as colunas 'min_date' e 'max_date'
dim_temp['min_date'] = min_date_str
dim_temp['max_date'] = max_date_str
dim_temp = dim_temp.drop_duplicates()
# Quantidade de registos únicos criados
len(dim_temp)

In [9]:
# Lista city
city_list = list(dim_temp['city_code'].unique())
# lista armazenar temperturas das city
from datetime import datetime, timedelta
city_temp_list = []
import requests
for city in city_list:
    row = dim_temp[dim_temp['city_code']== city ]
    latitude = row['latitude'].values[0]
    longitude = row['longitude'].values[0]
    start_date = row['min_date'].values[0]
    end_date = row['max_date'].values[0]
    # Adicionar um dia à variável end_date
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    end_date = end_date + timedelta(days=1)
    end_date = end_date.strftime('%Y-%m-%d')
# API
    link = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}1&start_date={start_date}&end_date={end_date}&daily=temperature_2m_mean&timeformat=unixtime&timezone=Europe%2FLondon"
    req = requests.get(link)
    dict = req.json()
    selct_dict = dict['daily']
    city_temp_list.append(selct_dict)

In [15]:
# Transformar em dataframes as temperaturas
df_city_temp_list = []
for i in range(len(city_temp_list)):
    df = pd.DataFrame(city_temp_list[i])
    # Adicionar coluna city_code
    df.insert(0,'city_code',city_list[i])
# Converter Unix Time para formato de data
    df['date'] = pd.to_datetime(df['time'], unit='s').dt.strftime('%Y-%m-%d')
    df.drop('time', axis=1, inplace=True)
    df_city_temp_list.append(df)
# Tranformar as listas de df em df único
combined_df = pd.concat(df_city_temp_list, ignore_index=True)
combined_df['date'] = pd.to_datetime(combined_df['date'])
# Adicionar temperature ao dataframe principal
df_concatenado1 = df_concatenado.copy()
# Converter a coluna "date" para datetime
df_concatenado1['date'] = pd.to_datetime(df_concatenado1['date'])
df_concatenado1 = df_concatenado1.merge(combined_df, on = ['city_code', 'date'], how = 'left')
df_concatenado1.rename(columns={'temperature_2m_mean': 'temperature'}, inplace=True)
df_concatenado1.head()

,store_id,city_code,date,latitude,longitude,revenue,temperature
0,S0002,Adana,2017-01-02,36.98636,35.325286,171.59,6.6
1,S0002,Adana,2017-01-03,36.98636,35.325286,178.10,6.7
2,S0002,Adana,2017-01-04,36.98636,35.325286,303.89,7.4
3,S0002,Adana,2017-01-05,36.98636,35.325286,274.65,9.3
4,S0002,Adana,2017-01-06,36.98636,35.325286,193.81,9.0


## Agrupar registos em granularidade semanal

In [16]:
# Voltar a criar dicionario de dataframes por store_id
df_concatenado2 = df_concatenado1.copy()
# Criar uma lista das lojas únicas
stores = df_concatenado2['store_id'].unique().tolist()
# Criar um dicionário para armazenar os dataframes separados
df_dict3 = {}
# Loop para criar os dataframes separados
for store in stores:
    # Filtrar o dataframe com base no valor da loja
    df_store = df_concatenado2[df_concatenado2['store_id'] == store].copy()
    # Redefinir o índice do dataframe
    df_store = df_store.reset_index(drop=True)
    # Adicionar o dataframe ao dicionário usando o nome da store_id como chave
    df_dict3[store] = df_store
# Criar um dicionario para armazenar os dataframes com granularidade semanal
import numpy as np
df_dict_week = {}
for key, df in df_dict3.items():
    # Converter coluna 'date' para tipo datetime
    df['date'] = pd.to_datetime(df['date'])
    # Definir a coluna 'date' como o índice do DataFrame
    df = df.set_index('date')
    # Resample para dados semanais e soma dos valores
    df_weekly = df.groupby(['store_id']).resample('W').agg({ 'revenue': np.sum, 'temperature': np.mean }).reset_index()
    df_weekly = df_weekly.rename(columns={'revenue': 'summed_revenue', 'temperature': 'mean_week_temperature'  })
    # Retirar ultimo registo uma vez que não existem 7 dias completos de dados
    df_weekly = df_weekly.groupby('store_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
    df_dict_week[key] = df_weekly
# Ver elementos da lista
len(df_dict_week)

63

## Criação de nova variável season

Primavera: Março-Abril-Maio.

Verão: Junho- Julho-Agosto.

Outono: Setembro-Outubro-Novembro.

Inverno: Dezembro-Janeiro-Fevereiro.

In [18]:
df_dict_week1 = df_dict_week.copy()
for key, df in df_dict_week1.items():
# Criar uma coluna "season" no dataframe
    df['season'] = ''
# Mapear as datas para as estações correspondentes
    df.loc[df['date'].dt.month.isin([3, 4, 5]), 'season'] = 'spring'
    df.loc[df['date'].dt.month.isin([6, 7, 8]), 'season'] = 'summer'
    df.loc[df['date'].dt.month.isin([9, 10, 11]), 'season'] = 'autumn'
    df.loc[df['date'].dt.month.isin([12, 1, 2]), 'season'] = 'winter'
    df_dict_week1[key] = df
# Ver elementos do dicionario
len(df_dict_week1)

63

## Criação de variável holiday

Feriados Fixos

Data	Feriado	Nome local

30 ago.	Dia da Vitória	Zafer Bayramı

9 set.	Dia da Libertação de İzmir	Kurtuluş Savaşı

19 set.	Dia dos Veteranos	Gaziler Günü

29 out.	Dia da República	Cumhuriyet Bayramı

In [19]:
from datetime import date, timedelta
# Lista dos feriados fixos nos anos dos nossos dados
feriados_fixos = [
    (date(2017, 8, 30), "Dia da Vitória"),
    (date(2017, 9, 9), "Dia da Libertação de İzmir"),
    (date(2017, 9, 19), "Dia dos Veteranos"),
    (date(2017, 10, 29), "Dia da República"),
    (date(2018, 8, 30), "Dia da Vitória"),
    (date(2018, 9, 9), "Dia da Libertação de İzmir"),
    (date(2018, 9, 19), "Dia dos Veteranos"),
    (date(2018, 10, 29), "Dia da República"),
    (date(2019, 8, 30), "Dia da Vitória"),
    (date(2019, 9, 9), "Dia da Libertação de İzmir"),
    (date(2019, 9, 19), "Dia dos Veteranos"),
    (date(2019, 10, 29), "Dia da República")
]
# Função para verificar se existe um feriado entre as datas
def verificar_feriado(row):
    for feriado, nome in feriados_fixos:
        if row['date'].date() <= feriado <= (row['date'] + timedelta(days=6)).date():
            return 1
    return 0
df_dict_week2 = df_dict_week1.copy()
#loop
for key, df in df_dict_week2.items():
    # Aplicar a função a cada linha do DataFrame e adicionar a nova coluna
    df['holiday'] = df.apply(verificar_feriado, axis=1)
    df_dict_week1[key] = df  
# Ver elementos do dicionario
len(df_dict_week2)

63

## Criação de variável number month

In [20]:
## Criação de variável month
df_dict_week3 = df_dict_week2.copy()
#loop
for key, df in df_dict_week3.items():
    # 
    df['number_month'] = df['date'].dt.month
    df_dict_week3[key] = df 
# Ver elementos do dicionario
len(df_dict_week3)

63

## Criação de variável number week

In [30]:
# Concatenar os DataFrames do  em um único DataFrame
combined_df = pd.concat(df_dict_week3.values())
import datetime
# Função para determinar o numero da semana
def week_of_year(date):
    date_str = date.strftime("%Y-%m-%d")  # Convert Timestamp to string
    data_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    week_year = data_obj.isocalendar()[1]
    return week_year
combined_df['number_week'] = combined_df['date'].apply(week_of_year)
combined_df.head()

,store_id,date,summed_revenue,mean_week_temperature,season,holiday,number_month,number_week
0,S0002,2017-01-08,1813.24,7.828571,winter,0,1,1
1,S0002,2017-01-15,1896.91,7.071429,winter,0,1,2
2,S0002,2017-01-22,7404.16,7.142857,winter,0,1,3
3,S0002,2017-01-29,2078.39,5.614286,winter,0,1,4
4,S0002,2017-02-05,1745.47,4.714286,winter,0,2,5


In [33]:
# Criar csv para consulta quando necessário
combined_df.to_csv(r"C:\Users\Egomes\Desktop\PG_Analytics_Data_science_empresarial\Isla_gaia\14-ProjetoII\projetoII\dataStaging\grainWeekMultipleVariables.csv", index=False)
print('Exportação concluida!')

Exportação concluida!
